In [1]:
# !pip install dagshub


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Now you can import from src
from src.data_utils import load_and_process_taxi_data

import pandas as pd

import dagshub
dagshub.init(repo_owner="gourimenon8", repo_name="sp25_taxi", mlflow=True)

Accessing as gourimenon8

Initialized MLflow to track repo "gourimenon8/sp25_taxi"

Repository gourimenon8/sp25_taxi initialized!

In [3]:
rides1 = load_and_process_taxi_data(year=2022)
rides2=(load_and_process_taxi_data(year=2023))
rides = pd.concat([rides1,rides2],ignore_index=True)

File already exists for 2022-01.
Loading data for 2022-01...
Total records: 2,463,931
Valid records: 2,415,141
Records dropped: 48,790 (1.98%)
Successfully processed data for 2022-01.
File already exists for 2022-02.
Loading data for 2022-02...
Total records: 2,979,431
Valid records: 2,921,118
Records dropped: 58,313 (1.96%)
Successfully processed data for 2022-02.
File already exists for 2022-03.
Loading data for 2022-03...
Total records: 3,627,882
Valid records: 3,551,986
Records dropped: 75,896 (2.09%)
Successfully processed data for 2022-03.
File already exists for 2022-04.
Loading data for 2022-04...
Total records: 3,599,920
Valid records: 3,522,113
Records dropped: 77,807 (2.16%)
Successfully processed data for 2022-04.
File already exists for 2022-05.
Loading data for 2022-05...
Total records: 3,588,295
Valid records: 3,509,056
Records dropped: 79,239 (2.21%)
Successfully processed data for 2022-05.
File already exists for 2022-06.
Loading data for 2022-06...
Total records: 3,55

In [4]:
temp_rides = rides[rides["pickup_location_id"] == 43]

In [5]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(temp_rides)
ts_data.head()

C:\gouri\MLops\sp25_taxi-main\src\data_utils.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides["pickup_hour"] = rides["pickup_datetime"].dt.floor("h")


,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4


In [6]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4
...,...,...,...
17515,2023-12-31 19:00:00,43,55
17516,2023-12-31 20:00:00,43,72
17517,2023-12-31 21:00:00,43,50
17518,2023-12-31 22:00:00,43,28


In [7]:
ts_data = ts_data.drop(columns=["pickup_location_id"])

In [8]:
print(ts_data.head())


          pickup_hour  rides
0 2022-01-01 00:00:00     96
1 2022-01-01 01:00:00     60
2 2022-01-01 02:00:00     22
3 2022-01-01 03:00:00      8
4 2022-01-01 04:00:00      4


In [9]:
# !pip install pmdarima
from pmdarima import auto_arima

# Find best ARIMA(p, d, q)
model_auto = auto_arima(ts_data["rides"], seasonal=False, stepwise=True, trace=True)

# Print the best parameters
print(model_auto.summary())


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=162977.313, Time=4.68 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=210122.120, Time=0.28 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=165205.279, Time=0.34 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=191455.692, Time=1.97 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=164599.482, Time=5.13 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=164969.807, Time=5.63 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=inf, Time=27.01 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=162484.750, Time=11.06 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=163384.864, Time=4.27 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=162468.514, Time=6.74 sec
 ARIMA(4,0,3)(0,0,0)[0]             : AIC=159080.981, Time=16.08 sec
 ARIMA(4,0,2)(0,0,0)[0]             : AIC=162619.471, Time=5.69 sec
 ARIMA(5,0,3)(0,0,0)[0]             : AIC=162610.899, Time=16.71 sec
 ARIMA(4,0,4)(0,0,0)[0]             : AIC=156798.051, Time=25.67 sec
 ARIMA(

In [16]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the best ARIMA model (Example: ARIMA(4, 0, 4) if suggested)
model_arima = ARIMA(ts_data["rides"], order=(4, 0, 4))  # Replace with best (p, d, q)
arima_result = model_arima.fit()

# Print model summary
print(arima_result.summary())

C:\Users\gouri\.conda\envs\mlops_project1\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  rides   No. Observations:                17520
Model:                 ARIMA(4, 0, 4)   Log Likelihood              -78008.756
Date:                Fri, 07 Mar 2025   AIC                         156037.512
Time:                        14:59:42   BIC                         156115.223
Sample:                             0   HQIC                        156063.100
                              - 17520                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.4145      0.761     96.461      0.000      71.923      74.906
ar.L1          1.6857      0.007    244.919      0.000       1.672       1.699
ar.L2          0.2286      0.007     34.941      0.0

In [17]:
from sklearn.metrics import mean_absolute_error

# Split data into train (80%) and test (20%)
train_size = int(len(ts_data) * 0.8)
train, test = ts_data.iloc[:train_size], ts_data.iloc[train_size:]

# Fit ARIMA on training data
model_arima = ARIMA(train["rides"], order=(4, 0, 4))  # Use optimal (p,d,q)
arima_result = model_arima.fit()

# Forecast for test set
forecast = arima_result.forecast(steps=len(test))

# Compute MAE
mae = mean_absolute_error(test["rides"], forecast)
print("Mean Absolute Error (MAE) for ARIMA:", mae)


C:\Users\gouri\.conda\envs\mlops_project1\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Mean Absolute Error (MAE) for ARIMA: 25.450375690190363


In [18]:
import mlflow
mlflow.set_experiment("model_arima")

# Start an MLflow run
with mlflow.start_run():
        mlflow.log_metric("mean_absolute_error", mae)

🏃 View run unleashed-yak-519 at: https://dagshub.com/gourimenon8/sp25_taxi.mlflow/#/experiments/9/runs/15e20eeadcbf4d628de839c2a76a92c4
🧪 View experiment at: https://dagshub.com/gourimenon8/sp25_taxi.mlflow/#/experiments/9
